In [1]:
import os
import time
from typing import Any, TypeAlias

import ibis
import pandas as pd
from dotenv import load_dotenv
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from pandas import DataFrame
from rich.pretty import pprint
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm.decl_api import DeclarativeAttributeIntercept

# Connect to and Query Database
### Approaches Used
1. `ibis` framework ([ibis website](https://ibis-project.org/))
1. `flask-sqlalchemy` ([flask-sqlalchemy website](https://flask-sqlalchemy.palletsprojects.com/en/3.1.x/) and [sqlalchemy website](https://www.sqlalchemy.org/))

Both approaches are timed per iteration and across the entire for loop that loops over all of the tables in the database.

### Database URI

In [2]:
# Create environment variables from .env.
load_dotenv()

# Host, port and database values.
db = os.environ.get("PGDATABASE")
host = os.environ.get("PGHOST")
port = os.environ.get("PGPORT")

# User credentials.
user = os.environ.get("PGUSER")
pwd = os.environ.get("PGPASSWORD")

# Build database URI.
db_uri = f"postgresql://{user}:{pwd}@{host}:{port}/{db}"

## Retrieve Tables
### Using `ibis`
#### Helper Function

In [3]:
def retrieve_tables_ibis(
    table_names: list[str], connection: Any
) -> dict[str, DataFrame]:
    """Retrieve all database tables using ibis framework."""
    tables_ibis = {}
    total_start = time.perf_counter()
    for name in table_names:
        start = time.perf_counter()
        # Retrieve the ibis table.
        table = connection.table(name)
        # Convert ibis table to pandas DataFrame.
        tables_ibis[name] = table.to_pandas()
        iteration_time = time.perf_counter() - start
        print(f"'{name}' table iteration: {iteration_time:.2f}s")
    execution_time = time.perf_counter() - total_start
    print(f"\ntotal execution time of `for` loop: {execution_time:.2f}")
    return tables_ibis

#### Retrieval

In [4]:
# Create connection to the database.
con = ibis.postgres.connect(url=db_uri)
table_names = con.list_tables()
tables_ibis = retrieve_tables_ibis(table_names=table_names, connection=con)

'champions' table iteration: 3.74s
'playoff_records' table iteration: 3.68s
'playoff_teams' table iteration: 1.46s
'playoff_teams_long' table iteration: 3.70s
'playoffs_advanced' table iteration: 3.71s
'playoffs_clutch' table iteration: 3.74s
'playoffs_misc' table iteration: 3.64s
'playoffs_traditional' table iteration: 3.79s
'season_records' table iteration: 3.69s
'seasons' table iteration: 1.46s
'teams' table iteration: 1.43s
'teams_advanced' table iteration: 3.72s
'teams_clutch' table iteration: 3.82s
'teams_misc' table iteration: 3.68s
'teams_traditional' table iteration: 3.74s

total execution time of `for` loop: 49.02


#### Check Results

In [5]:
tables_ibis.keys()

dict_keys(['champions', 'playoff_records', 'playoff_teams', 'playoff_teams_long', 'playoffs_advanced', 'playoffs_clutch', 'playoffs_misc', 'playoffs_traditional', 'season_records', 'seasons', 'teams', 'teams_advanced', 'teams_clutch', 'teams_misc', 'teams_traditional'])

In [6]:
tables_ibis["teams_traditional"].head()

,SEASON,TEAM,WIN%,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,2024,Boston Celtics,1.0,113.5,41.0,86.0,47.7,14.0,39.0,35.9,...,39.0,50.5,19.0,14.0,6.5,8.5,3.0,20.5,19.0,6.0
1,2024,Dallas Mavericks,1.0,125.5,45.5,95.0,47.9,15.0,41.5,36.1,...,33.5,44.0,22.0,10.5,5.5,2.5,4.5,17.5,25.5,6.0
2,2024,Denver Nuggets,1.0,113.5,44.5,89.5,49.7,14.0,35.5,39.4,...,33.0,40.5,28.0,15.5,10.0,7.5,5.5,16.5,20.5,8.0
3,2024,Indiana Pacers,1.0,143.0,56.0,107.0,52.3,20.0,43.0,46.5,...,41.0,51.0,38.0,12.0,10.0,8.0,9.0,23.0,11.0,23.0
4,2024,Milwaukee Bucks,1.0,118.0,41.0,81.0,50.6,11.0,30.0,36.7,...,35.0,43.0,20.0,14.0,9.0,4.0,4.0,23.0,28.0,1.0


### Using `flask-sqlalchemy`
#### Flask App Preliminaries

In [7]:
class Base(DeclarativeBase):
    pass


# Create the `db` object based on a DeclarativeBase model.
db = SQLAlchemy(model_class=Base)

app = Flask(__name__)
# Configure the database URI on the `app` object.
app.config["SQLALCHEMY_DATABASE_URI"] = db_uri

#### Database Reflection and ORM Mapped Classes

In [8]:
with app.app_context():
    # Initialize the app with the extension.
    db.init_app(app)
    # Gather metadata by reflecting the database.
    db.reflect()

# Build the ORM classes using the metadata collected by `db.reflect()`. The
# `playoff_teams` table is left out due to its lack of primary key, see
# `playoff_teams_long` for the same information.


class Teams(db.Model):
    __table__ = db.metadata.tables["teams"]


class Seasons(db.Model):
    __table__ = db.metadata.tables["seasons"]


class SeasonRecords(db.Model):
    __table__ = db.metadata.tables["season_records"]


class PlayoffRecords(db.Model):
    __table__ = db.metadata.tables["playoff_records"]


class TeamsTraditional(db.Model):
    __table__ = db.metadata.tables["teams_traditional"]


class TeamsAdvanced(db.Model):
    __table__ = db.metadata.tables["teams_advanced"]


class TeamsMisc(db.Model):
    __table__ = db.metadata.tables["teams_misc"]


class TeamsClutch(db.Model):
    __table__ = db.metadata.tables["teams_clutch"]


class PlayoffsTraditional(db.Model):
    __table__ = db.metadata.tables["playoffs_traditional"]


class PlayoffsAdvanced(db.Model):
    __table__ = db.metadata.tables["playoffs_advanced"]


class PlayoffsMisc(db.Model):
    __table__ = db.metadata.tables["playoffs_misc"]


class PlayoffsClutch(db.Model):
    __table__ = db.metadata.tables["playoffs_clutch"]


class PlayoffTeamsLong(db.Model):
    __table__ = db.metadata.tables["playoff_teams_long"]


class Champions(db.Model):
    __table__ = db.metadata.tables["champions"]

#### Helper Function: Get ORM Mapped Class 

In [9]:
MappedClass: TypeAlias = DeclarativeAttributeIntercept


def get_mapped_class(table_name: str, global_vars: dict[str, Any]) -> MappedClass:
    """Get the sqlalchemy ORM class for the given `table_name`."""
    name_list = table_name.split("_")
    name_list = [name.title() for name in name_list]
    class_name = "".join(name_list)
    return global_vars[class_name]

#### Dict of Mapped Classes

In [10]:
global_vars = globals()
mapped_classes = {}
for name in db.metadata.tables:
    if name != "playoff_teams":
        mapped_classes[name] = get_mapped_class(name, global_vars)

In [11]:
pprint(mapped_classes, indent_guides=False, expand_all=True)

{
    'seasons': <class '__main__.Seasons'>,
    'season_records': <class '__main__.SeasonRecords'>,
    'teams': <class '__main__.Teams'>,
    'playoff_records': <class '__main__.PlayoffRecords'>,
    'teams_traditional': <class '__main__.TeamsTraditional'>,
    'teams_advanced': <class '__main__.TeamsAdvanced'>,
    'teams_misc': <class '__main__.TeamsMisc'>,
    'teams_clutch': <class '__main__.TeamsClutch'>,
    'playoffs_traditional': <class '__main__.PlayoffsTraditional'>,
    'playoffs_advanced': <class '__main__.PlayoffsAdvanced'>,
    'playoffs_misc': <class '__main__.PlayoffsMisc'>,
    'playoffs_clutch': <class '__main__.PlayoffsClutch'>,
    'playoff_teams_long': <class '__main__.PlayoffTeamsLong'>,
    'champions': <class '__main__.Champions'>
}

#### Helper Function: Retrieve Tables Using `sqlalchemy`

In [12]:
MappedClass: TypeAlias = DeclarativeAttributeIntercept


def retrieve_tables_sqlalchemy(
    mapped_classes: dict[str, MappedClass], db: SQLAlchemy
) -> dict[str, DataFrame]:
    """Retrieve all database tables using sqlalchemy framework."""
    tables_sqlalchemy = {}
    total_start = time.perf_counter()
    for name, cls in mapped_classes.items():
        start = time.perf_counter()
        # Create a SQL query statement (the query is not executed).
        query = db.select(cls)
        # Use `pd.read_sql` to execute the query and return a DataFrame.
        tables_sqlalchemy[name] = pd.read_sql(sql=query, con=db.engine)
        iteration_time = time.perf_counter() - start
        print(f"'{name}' table iteration: {iteration_time:.2f}s")
    execution_time = time.perf_counter() - total_start
    print(f"\ntotal execution time of `for` loop: {execution_time:.2f}")
    return tables_sqlalchemy

#### Retrieval

In [13]:
with app.app_context():
    tables_sqlalchemy = retrieve_tables_sqlalchemy(mapped_classes=mapped_classes, db=db)

'seasons' table iteration: 0.17s
'season_records' table iteration: 0.21s
'teams' table iteration: 0.15s
'playoff_records' table iteration: 0.16s
'teams_traditional' table iteration: 0.26s
'teams_advanced' table iteration: 0.29s
'teams_misc' table iteration: 0.25s
'teams_clutch' table iteration: 0.32s
'playoffs_traditional' table iteration: 0.21s
'playoffs_advanced' table iteration: 0.27s
'playoffs_misc' table iteration: 0.16s
'playoffs_clutch' table iteration: 0.22s
'playoff_teams_long' table iteration: 0.15s
'champions' table iteration: 0.15s

total execution time of `for` loop: 2.99


The times for the `flask-sqlalchemy` approach are much faster and will be used in the app for retrieving data from the database.

#### Check Results

In [14]:
tables_sqlalchemy.keys()

dict_keys(['seasons', 'season_records', 'teams', 'playoff_records', 'teams_traditional', 'teams_advanced', 'teams_misc', 'teams_clutch', 'playoffs_traditional', 'playoffs_advanced', 'playoffs_misc', 'playoffs_clutch', 'playoff_teams_long', 'champions'])

In [15]:
tables_sqlalchemy["teams_traditional"].head()

,SEASON,TEAM,WIN%,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,2024,Boston Celtics,1.0,113.5,41.0,86.0,47.7,14.0,39.0,35.9,...,39.0,50.5,19.0,14.0,6.5,8.5,3.0,20.5,19.0,6.0
1,2024,Dallas Mavericks,1.0,125.5,45.5,95.0,47.9,15.0,41.5,36.1,...,33.5,44.0,22.0,10.5,5.5,2.5,4.5,17.5,25.5,6.0
2,2024,Denver Nuggets,1.0,113.5,44.5,89.5,49.7,14.0,35.5,39.4,...,33.0,40.5,28.0,15.5,10.0,7.5,5.5,16.5,20.5,8.0
3,2024,Indiana Pacers,1.0,143.0,56.0,107.0,52.3,20.0,43.0,46.5,...,41.0,51.0,38.0,12.0,10.0,8.0,9.0,23.0,11.0,23.0
4,2024,Milwaukee Bucks,1.0,118.0,41.0,81.0,50.6,11.0,30.0,36.7,...,35.0,43.0,20.0,14.0,9.0,4.0,4.0,23.0,28.0,1.0


## Retrieve Individual Table
### Using `flask-sqlalchemy`
#### Retrieval Example: `teams_traditional` Table

In [16]:
with app.app_context():
    query = (
        db.select(SeasonRecords, TeamsTraditional)
        .join(
            TeamsTraditional,
            (SeasonRecords.SEASON == TeamsTraditional.SEASON)
            & (SeasonRecords.TEAM == TeamsTraditional.TEAM),
        )
        .where(SeasonRecords.SEASON <= 2022)
    )
    teams_traditional = pd.read_sql(sql=query, con=db.engine)
    teams_traditional = teams_traditional.drop(columns=["SEASON_1", "TEAM_1"])

In [17]:
teams_traditional.head()

,SEASON,TEAM,GP,W,L,MIN,PLAYOFFS,CHAMPION,WIN%,PTS,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,2022,Phoenix Suns,82,64,18,48.1,True,False,0.780,114.8,...,35.5,45.3,27.4,12.9,8.6,4.4,4.0,19.9,18.9,7.5
1,2022,Memphis Grizzlies,82,56,26,48.2,True,False,0.683,115.6,...,35.0,49.2,26.0,13.2,9.8,6.5,6.0,19.8,19.8,5.7
2,2022,Golden State Warriors,82,53,29,48.1,True,True,0.646,111.0,...,35.7,45.5,27.1,14.9,8.8,4.5,3.9,21.0,18.0,5.5
3,2022,Miami Heat,82,53,29,48.4,True,False,0.646,110.0,...,33.9,43.7,25.5,14.6,7.4,3.2,4.0,20.5,20.6,4.5
4,2022,Dallas Mavericks,82,52,30,48.2,True,False,0.634,108.0,...,33.8,43.0,23.4,12.5,6.7,4.0,3.7,19.7,20.1,3.3


In [18]:
for c in tables_sqlalchemy["champions"].columns:
    print(c)

SEASON
TEAM
